In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
import os
import scipy.sparse

# Load Data

In [3]:
#**************
# with NaNs
# tX
#tX_path=os.path.join( os.getcwd(), "..", "data", "combined-data-normalized-with-dummy-vars-med-imput-NaNs.csv")
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-whitened-with-dummy-vars-with-NaNs.csv")
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-standardized-with-dummy-vars-with-NaNs.csv")
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-standardized-with-dummy-vars-with-NaNs-removed-corr.csv")
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-whitened-using-PCA-with-dummy-vars-with-NaNs.csv")
tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-pca-with-dummy-vars-med-imput-NaNs.csv")

# y
y_path=os.path.join( os.getcwd(), "..", "data", "y-labels.csv")


#**************
# no NaNs
# tX
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-standardized-with-dummy-vars-no-NaNs-removed-corr.csv")
# y
#y_path=os.path.join( os.getcwd(), "..", "data", "y-labels-no-NaNs.csv")

y = np.loadtxt(y_path)

######
tX_train = np.loadtxt(tX_path)
####### 
#tX_all = np.loadtxt(tX_path)
#tX_test = np.loadtxt(tX_test_path)
#tX_train = tX_all[0:y.shape[0],:]
#tX_test = tX_all[y.shape[0]:,:]


print(tX_train.shape)

(250000, 10)


In [4]:
tX = tX_train

In [5]:
tX_test_path=os.path.join( os.getcwd(), "..", "data", "PCA-transformed-test-data.csv")
tX_test = np.loadtxt(tX_test_path)
print(tX_test.shape)

(568238, 10)


In [6]:
ids_path=os.path.join( os.getcwd(), "..", "data", "ids.csv")
ids = np.loadtxt(ids_path)
print (ids.shape)

(568238,)


In [7]:
N_SAMPLES = tX.shape[0]
print(N_SAMPLES)
print(y.shape[0])
print(tX.shape)
print(tX_test.shape)
print(np.max(np.abs(tX)))
print(np.max(np.abs(tX_test)))


250000
250000
(250000, 10)
(568238, 10)
1.0
1.0


In [8]:
y = np.loadtxt(y_path)

y_colors = np.array(['b']*N_SAMPLES)
y_colors[y==0] = 'r'
print("Found {b} boson events out of {N} total events".format(b=np.sum(y), N=N_SAMPLES))
# If I understood correctly, 1 are bosons, i.e. boson events will be colored in blue

Found 85667.0 boson events out of 250000 total events


## Compute Polynomial Basis

In [9]:
def full_multivar_poly_basis_deg_two(x):
    n_feat = x.shape[1]
    n_samp = x.shape[0]

    #do a first iteration to avoid concatenating empty array problems
    temp0 = x[0,:]
    temp = x[0,:]
    temp = np.outer(temp0,temp)
    upper_tri_indices = np.triu_indices(n=temp.shape[0],m=temp.shape[1])
    temp = temp[upper_tri_indices]
        
    x_ret = temp
    
    for sample in range(1,n_samp):
        temp0 = x[sample,:].reshape(n_feat,1)
        temp = x[sample,:].reshape(n_feat,1)
#        print(temp.shape)
        temp = np.outer(temp0,temp)
        upper_tri_indices = np.triu_indices(n=temp.shape[0],m=temp.shape[1])
        temp = temp[upper_tri_indices]
        x_ret = np.r_[x_ret, temp]
        
    x_ret = x_ret.reshape(n_samp, int(x_ret.size/n_samp))
    return x_ret
        

In [10]:
def reduced_multivar_poly_basis_deg_n(x, n):
    if n < 3:
        return np.array([]).reshape(x.shape[0],0)
    return np.power(x,n)

In [11]:
def my_poly_basis(x,deg):
    assert(deg > 0)
    x_ret = np.c_[np.ones((x.shape[0],1)), x]
    if deg <= 1:
        return x_ret
    
    x_ret = np.c_[x_ret, full_multivar_poly_basis_deg_two(x)]
    
    if deg > 1 and deg <= 2:
        return x_ret
    
    for d in range(3,deg+1):
        x_ret = np.c_[x_ret, reduced_multivar_poly_basis_deg_n(x, d)]
        
    return x_ret

In [ ]:
## test
a = np.array([2,5,11,7,9,13]).reshape((2,3))
print(a)
print("----")

my_poly_basis(a,3)

In [12]:
#tX_poly = my_poly_basis(tX,5)
print(np.max(np.abs(tX)))
degree = 11
tX_poly = np.ones((tX.shape[0],1))
for deg in range(1,degree+1):
    tX_poly = np.c_[tX_poly, np.power(tX,deg)]

1.0


In [13]:
print(np.max(np.abs(tX_poly)))
print(tX_poly.shape)

1.0
(250000, 111)


# Logistic Regression

In [14]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return 1.0/(1.0+np.exp(-1.0*t))

In [15]:
def calculate_likelihood(y, tx, w):
    """compute the cost by negative log likelihood."""
    Xw = tx.dot(w)
#    print(Xw)
#    print(y.shape)

    LOG_part = np.log( 1 + np.exp( Xw ) )
    
#    print(LOG_part.shape)
    PROD_part = np.multiply(y.reshape(N_SAMPLES,1), Xw )
#    print(PROD_part.shape)
    a = LOG_part - PROD_part
    return np.sum( a )

In [16]:
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    p = sigmoid(tx.dot(w))
    loss = -1.0*( y.T.dot(np.log(p))  + (1.0-y).T.dot(np.log(1.0-p)) )
#    print(loss.shape)
    return loss

In [17]:
def calculate_gradient(y, tx, w,lambda_=0.):
    """compute the gradient of loss."""
    return tx.T.dot( sigmoid(tx.dot(w)) - y.reshape((tx.shape[0],1)) ) #+ lambda_*w

## Gradient Descent

In [18]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """

    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y,tx, w)
    w -= gamma*grad
    
    return loss, w

In [21]:
from helpers import de_standardize
from plots import visualization

def logistic_regression_gradient_descent_demo(y, tx,w_initial):
    # init parameters
    max_iter = 50000
    threshold = 1e-8
    gamma = 1e-5
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0], 1)), x]
    #w = np.zeros((tx.shape[1], 1))
    w = w_initial

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        # log info
        if iter % 25 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        if len(losses) > 1 and (np.abs(losses[-1] - losses[-2]))/np.abs(losses[-2]) < threshold:
            break
    return losses, w

In [22]:
w0 = np.random.rand(tX_poly.shape[1],1)
losses, w_star = logistic_regression_gradient_descent_demo(y, tX_poly,w0)
#print(w_star.shape)
#print(w_star)
w_star_gd = w_star

Current iteration=0, the loss=[ 201080.05215516]
Current iteration=25, the loss=[ 158224.67406076]
Current iteration=50, the loss=[ 156827.77179395]
Current iteration=75, the loss=[ 155542.48332711]
Current iteration=100, the loss=[ 154358.51124415]
Current iteration=125, the loss=[ 153266.46200085]
Current iteration=150, the loss=[ 152257.81743558]
Current iteration=175, the loss=[ 151324.88962884]
Current iteration=200, the loss=[ 150460.76276021]
Current iteration=225, the loss=[ 149659.22717123]
Current iteration=250, the loss=[ 148914.71099401]
Current iteration=275, the loss=[ 148222.21362253]
Current iteration=300, the loss=[ 147577.24371194]
Current iteration=325, the loss=[ 146975.7629288]
Current iteration=350, the loss=[ 146414.13565914]
Current iteration=375, the loss=[ 145889.08434206]
Current iteration=400, the loss=[ 145397.6499031]
Current iteration=425, the loss=[ 144937.15674726]
Current iteration=450, the loss=[ 144505.18181634]
Current iteration=475, the loss=[ 1440

In [ ]:
f = plt.figure()
ax=f.add_subplot(2,1,1)
ax.scatter(whit_tX[:,0],whit_tX[:,1], c=y_colors, alpha=0.1)

ax=f.add_subplot(2,1,2)
prediction = sigmoid(whit_tX[:,0:1].dot(w_star[1:2]))
prediction = prediction < 0.5
#ax.plot(prediction)
ax.scatter(whit_tX[:,0],whit_tX[:,1], c=prediction, alpha=0.1)



In [ ]:
#tX_pred = np.c_[np.ones((y.shape[0], 1)), tX]
tX_pred = tX_poly
prediction = sigmoid(tX_pred.dot(w_star_gd))
prediction = np.array( [ int(x) for x in (prediction < 0.5)] )

error = np.sum( np.abs( prediction - y ))

print(prediction[0:20])
print(y[0:20])
print(error)
print(error/N_SAMPLES)

## Newton's Method

In [ ]:
def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    S1 = sigmoid(tx.dot(w))
    S1 = S1.reshape((N_SAMPLES,1))
#    print(S1.shape)
    S2 = 1.0 - sigmoid(tx.dot(w))
#    print(S2.shape)
    S = np.multiply(S1,S2)
#    print(S.shape)
    S = scipy.sparse.spdiags(S[:,0], 0, N_SAMPLES, N_SAMPLES)
    return tx.T.dot(S.dot(tx))

In [ ]:
def calculate_inv_hessian(y, tx, txinv, w):
    S1 = sigmoid(tx.dot(w))
    S1 = S1.reshape((N_SAMPLES,1))
    S2 = 1.0 - sigmoid(tx.dot(w))
    S = np.multiply(S1,S2)
    Sinv = scipy.sparse.spdiags(1.0/S[:,0], 0, N_SAMPLES, N_SAMPLES)
    return txinv.dot(Sinv.dot(txinv.T))

In [ ]:
def logistic_regression(y, tx, w,lambda_,txinv=0):
    """return the loss, gradient, and hessian."""
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y,tx, w,lambda_)
    #hess = calculate_hessian(y, tx, w)
    hess = calculate_inv_hessian(y, tx, txinv, w)
    return loss, grad, hess

In [ ]:
def learning_by_newton_method(y, tx, w, gamma, lambda_,txinv=0):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    loss, grad, hess = logistic_regression(y,tx,w,lambda_,txinv)
    #w -= gamma * np.linalg.inv(hess).dot(grad);
    w -= gamma * hess.dot(grad)
    return loss, w

In [ ]:
def logistic_regression_newton_method_demo(y, tx, w_initial):
    # init parameters
    max_iter = 5000
    gamma = 1.0e-4
    threshold = 1e-8
    lambda_ = 0.9
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0], 1)), x] #CAREFUL: poly basis already puts in the ones
    w = w_initial

    txinv = np.linalg.inv(tx)
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, gamma, lambda_,txinv)
        # log info
        if iter % 25 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2])/np.abs(losses[-1]) < threshold:
            print("[Exit Condition Met]: Current iteration={i}, the loss={l}".format(i=iter, l=loss)) 
            break
    return losses, w

In [ ]:
#w0 = np.random.rand(tX_poly.shape[1]+1,1)
w0 = np.random.rand(tX_poly.shape[1],1)
#w0 = np.zeros((tX_poly.shape[1],1))
losses, w_star = logistic_regression_newton_method_demo(y, tX_poly, w0)

## Compute Predictions

In [23]:
#tX_pred = np.c_[np.ones((y.shape[0], 1)), tX]
tX_pred = tX_poly
prediction = sigmoid(tX_pred.dot(w_star_gd))
prediction = np.array( [ int(x) for x in (prediction > 0.5)] )

## Plot predictions

In [ ]:
f = plt.figure()
ax=f.add_subplot(2,2,1)
ax.scatter(tX[:,0],tX[:,1], c=y_colors, alpha=0.1)

ax=f.add_subplot(2,2,2)
ax.scatter(tX[:,2],tX[:,3], c=y_colors, alpha=0.1)

prediction_colors = np.array(['b']*N_SAMPLES)
prediction_colors[prediction==0] = 'r'

ax=f.add_subplot(2,2,3)
ax.scatter(tX[:,0],tX[:,1], c=prediction_colors, alpha=0.1)

ax=f.add_subplot(2,2,4)
ax.scatter(tX[:,2],tX[:,3], c=prediction_colors, alpha=0.1)

## Compute training error

In [24]:
error = np.sum(np.abs(prediction - y))
print(prediction)
print(y)
print(error)
print(error/N_SAMPLES)

[0 0 0 ..., 0 0 0]
[ 1.  0.  0. ...,  1.  0.  0.]
63724.0
0.254896


## Predict on test data

In [ ]:
print(np.max(np.abs(tX_test)))
degree = 7
tX_test_poly = np.ones((tX_test.shape[0],1))
for deg in range(1,degree+1):
    tX_test_poly = np.c_[tX_test_poly, np.power(tX_test,deg)]

In [ ]:
print(tX_test_poly.shape)
tX_pred = tX_test_poly
prediction = sigmoid(tX_pred.dot(w_star_gd))
prediction = np.array( [ int(x) for x in (prediction > 0.5)] )
print(prediction.shape)

In [ ]:
from proj1_helpers import *
sub_path=os.path.join( os.getcwd(), "..", "data", "submission2.csv")
to_submit = 2*prediction - 1 
create_csv_submission(ids, to_submit, sub_path)

In [ ]:
sub_old = np.loadtxt("../data/submission.csv", skiprows=1, delimiter=",")

In [ ]:
print(sub_old.shape)
print( np.sum( np.abs(sub_old[:,1] - prediction) ) )

In [ ]:
(0.15*568238 + 389236 )/568238